In [1]:
# Some dependencies to verify

! pip install pandas
! pip install ete3
! pip install biopython
! pip install Levenshtein

/home/phil/miniconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/phil/miniconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/phil/miniconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/phil/miniconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.4 MB/s eta 0:00:0000:0100:01


In [2]:
# Package imports

import pandas as pd
import numpy as np

import ete3
import Levenshtein
from Bio import SeqIO

/home/phil/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Global variables 

N_SEQS = 30

In [4]:
%%bash

# Make output, data directories if they don't exist
mkdir -p greengenes/output
mkdir -p greengenes/data

# Download data if it doesn't exist
if [ ! -f "greengenes/data/gg_13_5.fasta" ]; then
    wget https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5.fasta.gz
    gunzip gg_13_5.fasta.gz
    mv gg_13_5.fasta greengenes/data/
    rm gg_13_5.fasta.gz
fi

if [ ! -f "greengenes/data/gg_13_5_otus_99_annotated.tree" ]; then
    wget https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_otus_99_annotated.tree.gz
    gunzip gg_13_5_otus_99_annotated.tree.gz
    mv gg_13_5_otus_99_annotated.tree greengenes/data/
    rm gg_13_5_otus_99_annotated.tree.gz

    # Remove the first 4 lines of the tree file - only if first line is '[':
    if [ "$(head -n 1 greengenes/data/gg_13_5_otus_99_annotated.tree)" = "[" ]; then
        tail -n +5 greengenes/data/gg_13_5_otus_99_annotated.tree > greengenes/data/gg_13_5_otus_99_annotated.tree.tmp
        mv greengenes/data/gg_13_5_otus_99_annotated.tree.tmp greengenes/data/gg_13_5_otus_99_annotated.tree
    fi
fi


In [5]:
# Import tree - I had to remove the first 4 lines of the tree file to get it to work

tree = ete3.Tree("greengenes/data/gg_13_5_otus_99_annotated.tree", format=1, quoted_node_names=True)
tree.describe()

Number of leaf nodes:	203452
Total number of nodes:	406903
Rooted:	Yes
Most distant node:	3882796
Max. distance:	1.394070


In [6]:
# Example of how the taxonomic information is stored

node = tree.get_leaves()[0]
anc = node.get_ancestors()

print(node.name)
for node in anc:
    if node.name != '':
        print(node.name)

1018666
s__epidermidis
g__Staphylococcus
f__Staphylococcaceae
p__Firmicutes; c__Bacilli; o__Bacillales
k__Bacteria


In [7]:
# Read 16S table, get top N OTUs

otus = pd.read_csv("greengenes/data/ibd_16s_otu_table.csv", dtype={0: str})
otus = otus.set_index(otus.columns[0])
topN_otus = otus.sum(axis=1).sort_values(ascending=False).head(N_SEQS).index

In [8]:
# Check overlap between top 10 OTUs and tree

otus_set = set(topN_otus)
tree_set = set([node.name for node in tree.get_leaves()])

print(len(otus_set.intersection(tree_set))) # Should be 10

30


In [9]:
# Filter tree to only include top 10 OTUs - branch lengths inferred from tree

topN_tree = tree.copy()
topN_tree.prune(topN_otus)
topN_tree.describe()
topN_tree.write(format=1, outfile=f"greengenes/output/top{N_SEQS}_tree.tree")

Number of leaf nodes:	30
Total number of nodes:	59
Rooted:	Yes
Most distant node:	181155
Max. distance:	0.124740


In [10]:
# Get 16S sequences and pairwise distances for top 10 OTUs as well

# First, index the greengenes fasta file
gg = SeqIO.index("greengenes/data/gg_13_5.fasta", "fasta")

# Get sequences for top 10 OTUs
topN_seqs = [gg[otu] for otu in topN_otus]

# Save sequences to fasta
SeqIO.write(topN_seqs, f"greengenes/output/top{N_SEQS}_seqs.fasta", "fasta")

# Get distances
topN_dists = np.zeros((N_SEQS, N_SEQS), dtype=int)
for i in range(N_SEQS):
    for j in range(i + 1, N_SEQS):
        topN_dists[i, j] = topN_dists[j, i] = Levenshtein.distance(
            str(topN_seqs[i].seq), str(topN_seqs[j].seq)
        )

print(topN_dists)

# Save pairwise distances to csv
pd.DataFrame(topN_dists, index=topN_otus, columns=topN_otus).to_csv(
    f"greengenes/output/top{N_SEQS}_dists.csv"
)

[[  0 555 481 172 595 389 191 549 538 209 196 403 539 408 176 505 243 437
  392 426 411 441 430 245 184 109 408 293 479 512]
 [555   0 424 508 533 491 522 392 394 525 553 454 445 479 529 344 550 530
  489 459 478 440 511 545 512 528 446 516 365 473]
 [481 424   0 476 294 178 477 271 314 470 480 365 378 481 467 378 481 440
  200 329 264 287 402 483 472 474 436 455 355 400]
 [172 508 476   0 607 415 119 516 495 173 151 386 498 360 145 504 255 437
  408 381 412 432 437 222 101 139 424 316 479 502]
 [595 533 294 607   0 349 609 368 409 599 576 539 439 534 587 455 611 569
  348 513 435 454 563 609 605 588 571 586 477 422]
 [389 491 178 415 349   0 418 363 388 389 415 279 443 535 385 442 391 342
   60 273 139 286 313 387 415 383 333 362 411 434]
 [191 522 477 119 609 418   0 504 498 201 108 382 518 379 172 515 256 447
  417 376 411 440 442 265  58 153 431 305 491 503]
 [549 392 271 516 368 363 504   0 150 516 535 303 327 435 522 377 544 509
  372 310 376 297 444 531 494 532 485 485 413 445]
